# Track biological samples

```{note}

This guide assumes no custom schema module, and works without a `Biosample` ORM.

This is enough for certain drylab operations! If you'd like to track wetlab operations, you'll need custom schema.

```

In [ ]:
!lamin init --storage ./test-samples --schema bionty

In [ ]:
import lamindb as ln
import lnschema_bionty as lb

ln.settings.verbosity = 3  # show hints

Samples, i.e., metadata associated with observations, are linked with the same approach post-ingestion.

In [ ]:
ln.File(ln.dev.datasets.file_mini_csv(), description="My mini dataset").save()

Let's first query an RNA-seq dataset stored as an `.csv` file.

In [ ]:
file = ln.select(ln.File, suffix=".csv").first()

In [ ]:
file

For instance, let's annotate a scRNA-seq dataset with its readout type (scRNA-seq), the tissue, and the species.

## ExperimentalFactor

In [ ]:
efo_bt = lb.ExperimentalFactor.bionty()  # equals to bionty.ExperimentalFactor()
efo_bt_lookup = efo_bt.lookup()

In [ ]:
efo_bt_lookup.single_cell_rna_sequencing

In [ ]:
efo_record = lb.ExperimentalFactor(efo_bt_lookup.single_cell_rna_sequencing)

efo_record.save()

### CellType

In [ ]:
celltype_bt = lb.CellType.bionty()  # equals to bionty.CellType()
celltype_bt_lookup = celltype_bt.lookup()

In [ ]:
celltype_bt_lookup.cd8_positive_alpha_beta_cytokine_secreting_effector_t_cell

In [ ]:
celltype_record = lb.CellType(
    celltype_bt_lookup.cd8_positive_alpha_beta_cytokine_secreting_effector_t_cell
)

In [ ]:
celltype_record.save()

## Link against file

Link metadata records against the data object:

In [ ]:
file.experimental_factors.add(efo_record)

In [ ]:
file.cell_types.add(celltype_record)

In [ ]:
file.describe()

## Query file from linked metadata

In [ ]:
efos = lb.ExperimentalFactor.lookup()

In [ ]:
ln.File.select(experimental_factors=efos.single_cell_rna_sequencing).df()

In [ ]:
cell_types = lb.CellType.lookup()

In [ ]:
ln.File.select(
    cell_types=cell_types.cd8_positive_alpha_beta_cytokine_secreting_effector_t_cell
).df()

In [ ]:
!lamin delete test-samples
!rm -r ./test-samples